<h1 align="center">PINCER manuscript tables</h1>

Pharmacist-led information technology intervention (PINCER) is an effective way to reduce errors in prescription and medication monitoring which are responsible for a large proportion of adverse drug events. This is achieved through the monitoring of 14 PINCER indicators.

The purpose of this notebook is to generate the tables required for PINCER manuscript.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpllimg
import matplotlib as mpl
import os
import json
from IPython.display import HTML, display, Markdown
mpl.rcParams['figure.dpi']= 200

def show_image(path):
    img=mpllimg.imread(path)
    img_plot = plt.imshow(img)
    plt.axis('off')
    plt.show()

def counts_table_read(folder):
    df = pd.read_csv(f"{folder}/demographics_summary.csv", index_col=False)
    df.columns = ['Attribute', 'Category', 'count', '%']
    # df.rename(columns={'Unnamed: 0': 'Attribute'}, inplace=True)
    # df.rename(columns={'Unnamed: 1': 'Category'}, inplace=True)
    df.set_index(["Attribute", "Category"], inplace=True)
    df.sort_index(inplace=True)
    return df
    
def rename_index_categories(df, attribute, replacement_dict):
    index = df.index.values
    for old, new in replacement_dict.items():
        for i, item in enumerate(index):
            if item == (attribute, old):
                index[i] = (attribute, new)
    df.index = pd.MultiIndex.from_tuples(index)
    return df

def reformat_data_tpp(df):
    high_level_ethnicities = {
        "1": "White",
        "2": "Mixed",
        "3": "South Asian",
        "4": "Black",
        "5": "Other",
    }
    imd_categories = {"0": "Missing",
                      "1": "Most deprived 1",
                      "5": "Least deprived 5"}
    region_categories = {"East": "East of England"}
    df = rename_index_categories(df, "ethnicity", high_level_ethnicities)
    df = rename_index_categories(df, "imd", imd_categories)
    df = rename_index_categories(df, "region", region_categories)
    return df


def reformat_data_emis(df):
    high_level_ethnicities = {
        "1": "White",
        "2": "Mixed",
        "3": "South Asian",
        "4": "Black",
        "5": "Other",
    }
    imd_categories = {"0": "Missing",
                      "1": "Most deprived 1",
                      "5": "Least deprived 5" }
    region_categories = {"East": "East of England"}
    df = rename_index_categories(df, "ethnicity", high_level_ethnicities)
    df = rename_index_categories(df, "imd", imd_categories)
    df = rename_index_categories(df, "region", region_categories)
    return df


## Population description

The first table to be generated describes the patient population used in the PINCER analysis, in terms of age group, sex, region, IMD (indices of multiple deprivation) and ethnicity. Counts and percentages are presented, for both TPP and EMIS.

In [2]:
tpp_population = counts_table_read( "../output" )


In [3]:

tpp_attribute_counts = tpp_population.reset_index().groupby('Attribute').sum()
tpp_total = tpp_attribute_counts.loc["sex","count"]

tpp_population_formatted = reformat_data_tpp(tpp_population)


In [4]:
emis_population = counts_table_read("../output")

emis_attribute_counts = emis_population.reset_index().groupby('Attribute').sum()
emis_total = emis_attribute_counts.loc["sex", "count"]

emis_population_formatted = reformat_data_emis(emis_population)



In [5]:

total_data = pd.merge(left=tpp_population_formatted.reset_index(),
                      right=emis_population_formatted.reset_index(),
                      how="outer",
                      left_on=['level_0', 'level_1'],
                      right_on=['level_0', 'level_1'])

total_data = total_data.assign( total_count = total_data.count_x + total_data.count_y )
total_data = total_data.assign( total_perc  = round( 100 * total_data.total_count / ( tpp_total + emis_total ), 2 ) )

total_data.rename( columns={'level_0': 'Attribute',
                           'level_1': 'Category',
                           'count_x': 'TPP count',
                           '%_x': 'TPP percentage',
                           'count_y': 'EMIS count',
                           '%_y': 'EMIS percentage' }, inplace=True )


## Cohort description

In [6]:
HTML(total_data.to_html(index=False))


Attribute,Category,TPP count,TPP percentage,EMIS count,EMIS percentage,total_count,total_perc
age_band,0-19,2817,12.26,2817,12.26,5634,12.26
age_band,20-29,2876,12.52,2876,12.52,5752,12.52
age_band,30-39,2903,12.63,2903,12.63,5806,12.63
age_band,40-49,3017,13.13,3017,13.13,6034,13.13
age_band,50-59,2894,12.59,2894,12.59,5788,12.59
age_band,60-69,2773,12.07,2773,12.07,5546,12.07
age_band,70-79,2875,12.51,2875,12.51,5750,12.51
age_band,80+,2704,11.77,2704,11.77,5408,11.77
age_band,missing,121,0.53,121,0.53,242,0.53
care_home_type,PC,1154,5.02,1154,5.02,2308,5.02
